In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
from gensim.models import word2vec
from janome.tokenizer import Tokenizer
import time
import re

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get("https://www.aozora.gr.jp/index_pages/person81.html")
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")

In [ ]:
element = driver.find_element_by_tag_name('body')
element = element.find_element_by_tag_name('ol')
elements = element.find_elements_by_tag_name('li')

In [ ]:
print(elements[58].text)

In [ ]:
page = elements[58].find_element_by_tag_name('a').click()
driver.implicitly_wait(10)
driver.title

In [ ]:
element = driver.find_element_by_css_selector('.download')

In [ ]:
elements = element.find_elements_by_tag_name('a')

In [ ]:
for a in elements:
    if a.text.endswith('html'):
        a.click()
driver.implicitly_wait(10)
time.sleep(3)

In [ ]:
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")

In [ ]:
text = ''.join(soup.select('.main_text')[0].findAll(text=True))

In [ ]:
t = Tokenizer()

In [ ]:
lines = text.split('\n')

In [ ]:
wakati = []
for line in lines:
    line = re.sub(r'（[^）]*）', '', line)
    tokens = t.tokenize(line)
    tmp = []
    for token in tokens:
        if token.part_of_speech.split(',')[0] in ['名詞', '形容詞', '動詞', '記号']:
            tmp.append(token.base_form)
    wakati.append(' '.join(tmp))
wakati = '\n'.join(wakati)

In [ ]:
wakati_filename = driver.title + '.wakati'
model_filename = driver.title + '.model'

In [ ]:
driver.quit()

In [ ]:
with open(wakati_filename, 'w', encoding='utf-8') as f:
    f.write(wakati)

In [ ]:
data = word2vec.LineSentence(wakati_filename)

In [ ]:
model = word2vec.Word2Vec(data, size=200, window=10, hs=1, min_count=2, sg=1)

In [ ]:
model.save(model_filename)

In [ ]:
model.wv.most_similar(positive=['ジョバンニ', 'カムパネルラ'], negative=['かなしい'])

In [ ]:
model.wv.most_similar(positive=['銀河', '鉄道'], negative=['星'])

In [ ]:
print(text)